In [1]:
import torch
import torch.nn.functional as F
import matplotlib.pyplot as plt # for making figures
%matplotlib inline

In [2]:
import os
os.chdir("../")

In [4]:
words = open('data/names.txt', 'r').read().splitlines()
words[:8]

['emma', 'olivia', 'ava', 'isabella', 'sophia', 'charlotte', 'mia', 'amelia']

In [3]:
len(words)

32033

In [5]:
## building vocab of chars & mapping to/from integers

chars = sorted(list(set(''.join(words))))
stoi = {s:i + 1 for i,s in enumerate(chars)}
stoi["."] = 0
itos = {i:s for s,i in stoi.items()}

In [6]:
print(itos)

{1: 'a', 2: 'b', 3: 'c', 4: 'd', 5: 'e', 6: 'f', 7: 'g', 8: 'h', 9: 'i', 10: 'j', 11: 'k', 12: 'l', 13: 'm', 14: 'n', 15: 'o', 16: 'p', 17: 'q', 18: 'r', 19: 's', 20: 't', 21: 'u', 22: 'v', 23: 'w', 24: 'x', 25: 'y', 26: 'z', 0: '.'}


In [20]:
## Building the data set

block_size = 3 ## context length: How many chars to take to predict the next one?

X, Y = [], []

for w in words[0:5]:
    print(w)
    context = [0] * block_size # -> 1st Iter [0, 0, 0]
    
    for ch in w + ".":

        ix = stoi[ch] ## Encoded values of character
        
        X.append(context) ## 1st Iter [0, 0, 0]
        Y.append(ix) ## 1st Iter [itos[ch]]

        print(''.join(itos[i] for i in context), '---->', itos[ix])
        context = context[1:] + [ix] ## Rolling Operation

X = torch.tensor(X)
Y = torch.tensor(Y)

emma
... ----> e
..e ----> m
.em ----> m
emm ----> a
mma ----> .
olivia
... ----> o
..o ----> l
.ol ----> i
oli ----> v
liv ----> i
ivi ----> a
via ----> .
ava
... ----> a
..a ----> v
.av ----> a
ava ----> .
isabella
... ----> i
..i ----> s
.is ----> a
isa ----> b
sab ----> e
abe ----> l
bel ----> l
ell ----> a
lla ----> .
sophia
... ----> s
..s ----> o
.so ----> p
sop ----> h
oph ----> i
phi ----> a
hia ----> .


In [22]:
X.shape, X.dtype, Y.shape, Y.dtype

(torch.Size([32, 3]), torch.int64, torch.Size([32]), torch.int64)

In [19]:
Y

tensor([ 5, 13, 13,  1,  0, 15, 12,  9, 22,  9,  1,  0,  1, 22,  1,  0])

## Projecting characters into 2 Dimensional Space

Number of Chars: 27

Dimensions: 2

In [23]:
C = torch.randn((27, 2))

### Accessing entries of Look-Up Matrix C

In [ ]:
## For 5 encoded char "e"
## itos[5]
C[5]

tensor([-0.3505, -1.7075])

In [31]:
## For 0 encoded char "a"
## itos[5]
C[0]

tensor([-1.3136, -1.6273])

In [28]:
## Is there a better way to index Look Up Matrix?
## Yes One Hot Encoding


F.one_hot(torch.tensor(5), num_classes=27).float() @ C


tensor([-0.3505, -1.7075])

In [36]:
C[[5, 0]]

tensor([[-0.3505, -1.7075],
        [-1.3136, -1.6273]])

In [37]:
C[torch.tensor([5, 0])]

tensor([[-0.3505, -1.7075],
        [-1.3136, -1.6273]])

In [30]:
F.one_hot(torch.tensor([5, 0]), num_classes=27).float() @ C

tensor([[-0.3505, -1.7075],
        [-1.3136, -1.6273]])

In [38]:
C[X]

tensor([[[-1.3136, -1.6273],
         [-1.3136, -1.6273],
         [-1.3136, -1.6273]],

        [[-1.3136, -1.6273],
         [-1.3136, -1.6273],
         [-0.3505, -1.7075]],

        [[-1.3136, -1.6273],
         [-0.3505, -1.7075],
         [-0.0719,  1.2150]],

        [[-0.3505, -1.7075],
         [-0.0719,  1.2150],
         [-0.0719,  1.2150]],

        [[-0.0719,  1.2150],
         [-0.0719,  1.2150],
         [-1.2015, -0.9261]],

        [[-1.3136, -1.6273],
         [-1.3136, -1.6273],
         [-1.3136, -1.6273]],

        [[-1.3136, -1.6273],
         [-1.3136, -1.6273],
         [-0.8307, -0.8465]],

        [[-1.3136, -1.6273],
         [-0.8307, -0.8465],
         [ 1.2490,  0.0284]],

        [[-0.8307, -0.8465],
         [ 1.2490,  0.0284],
         [-0.2628,  0.8860]],

        [[ 1.2490,  0.0284],
         [-0.2628,  0.8860],
         [-0.7497, -1.1101]],

        [[-0.2628,  0.8860],
         [-0.7497, -1.1101],
         [-0.2628,  0.8860]],

        [[-0.7497, -1